# Coulomb gauge transformation

### Simulation parameters

In [1]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 2      
N = 64   
tau_sim = 0.1     
DTS = 4     

# Glasma fields
su_group = 'su3'
Qs = 2        
ns = 50    
factor = 0.8        
g2mu = Qs / factor     
g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          		
mu = g2mu / g**2          	
ir = 0.1 * g**2 * mu         
uv = 10.0       

nevents = 1

### Environment variables

In [2]:
import os

# I need to add this line to ask resources from a specific GPU, which is free. Our GPU server has no queing system
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,4"

os.environ["MY_NUMBA_TARGET"] = "cuda"
# os.environ["MY_NUMBA_TARGET"] = "numba"
os.environ["PRECISION"] = "double"
os.environ['GAUGE_GROUP'] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
import curraun.lattice as latt
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

import curraun.su as su
import curraun.coulomb as coulomb

Using CUDA
Using SU(3)
Using double precision


### Simulate glasma

In [3]:
niter = 1
# alpha abelian should be 1/p^2_max 
# p_max = 2*pi/L in lattice units 2*pi/N
alpha = 1/(2*np.pi/N)**2

In [4]:
from numba.core.errors import NumbaPerformanceWarning
import warnings

warnings.simplefilter('ignore', category=NumbaPerformanceWarning)

In [5]:
import pickle
from tqdm import tqdm

# Simulation routine
# def simulate(): 

# Derived parameters
a = L / N
E0 = N / L * hbarc
DT = 1.0 / DTS
maxt = int(tau_sim / a * DTS)

# Initialize Glasma fields
s = core.Simulation(N, DT, g)
va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
initial.init(s, va, vb)

coul = coulomb.CoulombGaugeTransf(s, alpha)

# with tqdm(total=maxt) as pbar:
# for t in range(maxt):      
#TODO: evolve the glasma fields, at the moment consider only 1 evolution step

# Evolve Glasma fields
core.evolve_leapfrog(s)

coulomb.gauge_transform(coul, alpha)
delta0 = coul.delta0.copy()
# print(delta0)

# c = coul.c.copy()
# print(c)

# thetax = coul.thetax.copy()
# print(thetax)

# print('theta=', coul.theta)

# g1 = coul.g1.copy()
# print(g1)

ug0 = coul.ug0.copy()
print(ug0[1, 0, :])

ug1 = coul.ug1.copy()
print(ug1[1, 0, :])

# pbar.update(1)

if use_cuda:
    cuda.current_context().deallocations.clear()

# return ax

[ 0.92163634+0.16247409j -0.11953315+0.21930026j -0.22855192-0.09783592j
  0.11918451+0.22860275j  0.95647261-0.014839j    0.13508971-0.01505174j
  0.20720859-0.12161029j -0.13182339+0.07198447j  0.94831001-0.14291427j]
[ 0.96307996+0.21812744j  0.06798308-0.10717217j -0.04204003-0.08380019j
 -0.07686323-0.09296004j  0.89396636-0.4127579j   0.09901926+0.07810712j
  0.08727411-0.05226005j -0.0828849 +0.08647303j  0.96618304+0.20443816j]


### Run simulations

In [6]:
# Supress various horribly long warnings
import warnings
warnings.filterwarnings('ignore')